In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import math 

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Import xlsx file and store each sheet in to a df list
xl_file = pd.ExcelFile('./data.xlsx',)

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [3]:
# Data from each sheet can be accessed via key
keyList = list(dfs.keys())

In [4]:
# Data cleansing
for key, df in dfs.items():
    dfs[key].loc[:,'Confirmed'].fillna(value=0, inplace=True)
    dfs[key].loc[:,'Deaths'].fillna(value=0, inplace=True)
    dfs[key].loc[:,'Recovered'].fillna(value=0, inplace=True)
    dfs[key]=dfs[key].astype({'Confirmed':'int64', 'Deaths':'int64', 'Recovered':'int64'})
    # Change as China for coordinate search
    dfs[key]=dfs[key].replace({'Country/Region':'Mainland China'}, 'China')
    dfs[key]=dfs[key].replace({'Province/State':'Queensland'}, 'Brisbane')
    dfs[key]=dfs[key].replace({'Province/State':'New South Wales'}, 'Sydney')
    dfs[key]=dfs[key].replace({'Province/State':'Victoria'}, 'Melbourne')
    # Add a zero to the date so can be convert by datetime.strptime as 0-padded date
    dfs[key]['Last Update'] = '0' + dfs[key]['Last Update']
    dfs[key]['Date_last_updated'] = [datetime.strptime(d, '%m/%d/%Y %H:%M') for d in dfs[key]['Last Update']]

In [5]:
# Check 
dfs[keyList[0]].head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date_last_updated
0,Hubei,China,01/30/2020 21:30,5806,204,116,2020-01-30 21:30:00
1,Zhejiang,China,01/30/2020 21:30,537,0,9,2020-01-30 21:30:00
2,Guangdong,China,01/30/2020 21:30,393,0,11,2020-01-30 21:30:00
3,Henan,China,01/30/2020 21:30,352,2,3,2020-01-30 21:30:00
4,Hunan,China,01/30/2020 21:30,332,0,2,2020-01-30 21:30:00


In [6]:
# Add coordinates for each area in the list for the latest table sheet
# As there are limit for free account, we only call coordinates for the latest table sheet
from opencage.geocoder import OpenCageGeocode
key = 'b33700b33d0a446aa6e16c0b57fc82d1'  # get api key from:  https://opencagedata.com
geocoder = OpenCageGeocode(key)

list_lat = []   # create empty lists
list_long = []    
for index, row in dfs[keyList[0]].iterrows(): # iterate over rows in dataframe
    City = row['Province/State']
    State = row['Country/Region']

    if City:
        if City == 'Macau' or City == 'Hong Kong':
            query = str(City)+','+'China'
            results = geocoder.geocode(query)   
            lat = results[0]['geometry']['lat']
            long = results[0]['geometry']['lng']

            list_lat.append(lat)
            list_long.append(long)
        else:
            query = str(City)+','+str(State)
            results = geocoder.geocode(query)   
            lat = results[0]['geometry']['lat']
            long = results[0]['geometry']['lng']

            list_lat.append(lat)
            list_long.append(long)

    else:
        query = str(State)
        results = geocoder.geocode(query)   
        lat = results[0]['geometry']['lat']
        long = results[0]['geometry']['lng']

        list_lat.append(lat)
        list_long.append(long)
# create new columns from lists    
dfs[keyList[0]]['lat'] = list_lat   
dfs[keyList[0]]['lon'] = list_long

In [7]:
dfs[keyList[0]]

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date_last_updated,lat,lon
0,Hubei,China,01/30/2020 21:30,5806,204,116,2020-01-30 21:30:00,31.151725,112.878322
1,Zhejiang,China,01/30/2020 21:30,537,0,9,2020-01-30 21:30:00,29.000000,120.000000
2,Guangdong,China,01/30/2020 21:30,393,0,11,2020-01-30 21:30:00,23.135769,113.198269
3,Henan,China,01/30/2020 21:30,352,2,3,2020-01-30 21:30:00,34.000000,114.000000
4,Hunan,China,01/30/2020 21:30,332,0,2,2020-01-30 21:30:00,27.666209,111.748706
5,Jiangxi,China,01/30/2020 21:30,240,0,7,2020-01-30 21:30:00,28.000000,116.000000
6,Anhui,China,01/30/2020 21:30,237,0,3,2020-01-30 21:30:00,32.000000,117.000000
7,Chongqing,China,01/30/2020 21:30,206,0,1,2020-01-30 21:30:00,29.558571,106.549282
8,Shandong,China,01/30/2020 21:30,178,0,2,2020-01-30 21:30:00,36.000000,119.000000
9,Sichuan,China,01/30/2020 21:30,177,1,1,2020-01-30 21:30:00,30.500000,102.500000


In [8]:
# Save numbers into variables to use in the app
confirmedCases=dfs[keyList[0]]['Confirmed'].sum()
deathsCases=dfs[keyList[0]]['Deaths'].sum()
recoveredCases=dfs[keyList[0]]['Recovered'].sum()


In [9]:
# Construct new dataframe for line plot
DateList = []
ChinaList =[]
OtherList = []

for key, df in dfs.items():
    dfTpm = df.groupby(['Country/Region'])['Confirmed'].agg(np.sum)
    dfTpm = pd.DataFrame({'Code':dfTpm.index, 'Confirmed':dfTpm.values})
    dfTpm = dfTpm.sort_values(by='Confirmed', ascending=False).reset_index(drop=True)
    DateList.append(df['Date_last_updated'][0])
    ChinaList.append(dfTpm['Confirmed'][0])
    OtherList.append(dfTpm['Confirmed'][1:].sum())
    
df_confirmed = pd.DataFrame({'Date':DateList,
                             'Mainland China':ChinaList,
                             'Other locations':OtherList})    


In [10]:
df_confirmed.head()

,Date,Mainland China,Other locations
0,2020-01-30 21:30:00,9658,118
1,2020-01-30 11:00:00,8124,111
2,2020-01-29 21:00:00,7678,105
3,2020-01-29 14:30:00,6070,95
4,2020-01-29 13:30:00,6070,94


In [160]:
# Save numbers into variables to use in the app
latestDate=datetime.strftime(df_confirmed['Date'][0], '%b %d %Y %H:%M EST')
daysOutbreak=(df_confirmed['Date'][0] - datetime.strptime('12/31/2019', '%m/%d/%Y')).days

In [98]:
# Set up tick scale based on confirmed case number
tickList = list(np.arange(0, df_confirmed['Mainland China'].max()+1000, 1000))

# Create empty figure canvas
fig = go.Figure()
# Add trace to the figure
fig.add_trace(go.Scatter(x=df_confirmed['Date'], y=df_confirmed['Mainland China'],
                         mode='lines+markers',
                         name='Mainland China',
                         line=dict(color='#fe582c', width=2),
                         marker=dict(size=8),
                         text=[datetime.strftime(d, '%b %d %Y %H:%M EST') for d in df_confirmed['Date']],
                         hovertemplate='<b>%{text}</b><br></br>'+
                                       'Mainland China confirmed<br>'+
                                       '%{y} cases<br>'+
                                       '<extra></extra>'))
fig.add_trace(go.Scatter(x=df_confirmed['Date'], y=df_confirmed['Other locations'],
                         mode='lines+markers',
                         name='Other Region',
                         line=dict(color='#2cfec1', width=2),
                         marker=dict(size=8),
                         text=[datetime.strftime(d, '%b %d %Y %H:%M EST') for d in df_confirmed['Date']],
                         hovertemplate='<b>%{text}</b><br></br>'+
                                       'Other region confirmed<br>'+
                                       '%{y} cases<br>'+
                                       '<extra></extra>'))
# Customise layout
fig.update_layout(
    title=dict(
        text="<b>Confirmed Cases Timeline<b>",
        y=0.96, x=0.5, xanchor='center', yanchor='top',
        font=dict(size=20, color="#7fafdf", family="sans-serif")
    ),
    margin=go.layout.Margin(
        l=5,
        r=10,
        b=10,
        t=50,
        pad=0
    ),
    yaxis=dict(
        showline=True, linecolor='#272e3e',
        zeroline=False,
        gridcolor='#272e3e',
        gridwidth = .1,
        tickmode='array',
        # Set tick range based on the maximum number
        tickvals=tickList,
        # Set tick label accordingly
        ticktext=['{:.0f}k'.format(i/1000) for i in tickList]
    ),
#    yaxis_title="Confirmed case number",
    xaxis=dict(
        showline=True, linecolor='#272e3e',
        gridcolor='#272e3e',
        gridwidth = .1,
        zeroline=False
    ),
    xaxis_tickformat='%b %d',
    legend_orientation="h",
    plot_bgcolor='#151920',
    paper_bgcolor='#272e3e',
    font=dict(color='#7fafdf')
)

fig.show()

In [99]:
mapbox_access_token = "pk.eyJ1IjoicGxvdGx5bWFwYm94IiwiYSI6ImNqdnBvNDMyaTAxYzkzeW5ubWdpZ2VjbmMifQ.TXcBE-xg9BFdV2ocecc_7g"

textList=[]
for area, region in zip(dfs[keyList[0]]['Province/State'], dfs[keyList[0]]['Country/Region']):
    
    if type(area) is str:
        if region == "Hong Kong" or region == "Macau" or region == "Taiwan":
            textList.append(area)
        else:
            textList.append(area+', '+region)
    else:
        textList.append(region)

fig2 = go.Figure(go.Scattermapbox(
        lat=dfs[keyList[0]]['lat'],
        lon=dfs[keyList[0]]['lon'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            color='#ff1414',
            size=dfs[keyList[0]]['Confirmed'].tolist(), 
            sizemin=2,
            sizemode='area',
            sizeref=2.*max(dfs[keyList[0]]['Confirmed'].tolist())/(40.**2),
        ),
        text=textList,
        hovertext=['Comfirmed: {}<br>Death: {}<br>Recovered: {}'.format(i, j, k) for i, j, k in zip(dfs[keyList[0]]['Confirmed'],
                                                                                                    dfs[keyList[0]]['Deaths'],
                                                                                                    dfs[keyList[0]]['Recovered'])],
    
        hovertemplate = "<b>%{text}</b><br><br>" +
                        "%{hovertext}<br>" +
                        "<extra></extra>")
    
        )

fig2.update_layout(
    title=dict(
        text="<b>Latest Coronavirus Outbreak Map<b>",
        y=0.96, x=0.5, xanchor='center', yanchor='top',
        font=dict(size=20, color="#7fafdf", family="sans-serif")
    ),
    plot_bgcolor='#151920',
    paper_bgcolor='#272e3e',
    margin=go.layout.Margin(
        l=10,
        r=10,
        b=10,
        t=50,
        pad=40
    ),
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        style="mapbox://styles/plotlymapbox/cjvprkf3t1kns1cqjxuxmwixz",
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=29.538860, 
            lon=173.304781
        ),
        pitch=0,
        zoom=2
    )
)

fig2.show()

In [13]:
import dash
import dash_core_components as dcc
import dash_html_components as html 
import dash_bootstrap_components as dbc

In [172]:
app = dash.Dash(__name__, assets_folder='./assets/',
                meta_tags=[
                    {"name": "viewport", "content": "width=device-width, height=device-height, initial-scale=1.0"}
    ])

In [177]:
app.layout = html.Div(style={'backgroundColor':'#151920'},
    children=[
        html.Div(
            id="header",
            children=[
                html.P(id="logo", children=["Data source from ", html.A('JHU CSSE', href='https://docs.google.com/spreadsheets/d/1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w/htmlview?usp=sharing&sle=true#')]),
                html.H4(children="Wuhan Coronavirus (2019-nCoV) Outbreak Monitor"),
                html.P(
                    id="description",
                    children="On Dec 31, 2019, the World Health Organization (WHO) was informed of \
                    an outbreak of “pneumonia of unknown cause” detected in Wuhan City, Hubei Province, China – the \
                    seventh-largest city in China with 11 million residents. As of {}, there are over {} cases \
                    of 2019-nCoV confirmed globally.\
                    This dash board is developed to visualise and track the recent reported \
                    cases on a daily timescale.".format(latestDate, confirmedCases),
                ),
                html.P(
                    children="Last updated on {}".format(latestDate))
            ]        
        ),
        html.Div(style={'marginLeft':'1.5%','marginRight':'1.5%'},
                 children=[
                     html.Div(
                         children=[
                             html.Div(style={'width':'24.6%','backgroundColor':'#272e3e','display':'inline-block','marginRight':'10px'},
                                 children=[html.P(style={'textAlign':'center','fontSize':'3rem',
                                                         'fontWeight':'bold','color':'#ffffbf','padding':'1rem'},
                                                  children="Days Since Outbreak"),
                                           html.H3(style={'textAlign':'center','fontSize':'6rem',
                                                          'fontWeight':'bold','color':'#ffffbf'},
                                                  children='{}'.format(daysOutbreak))]),
                             html.Div(style={'width':'24.6%','backgroundColor':'#272e3e','display':'inline-block','marginRight':'10px'},
                                 children=[html.P(style={'textAlign':'center','fontSize':'3rem',
                                                         'fontWeight':'bold','color':'#d7191c','padding':'1rem'},
                                                  children="Confirmed Cases"),
                                           html.H3(style={'textAlign':'center','fontSize':'6rem',
                                                          'fontWeight':'bold','color':'#d7191c'},
                                                  children='{}'.format(confirmedCases))]),
                             html.Div(style={'width':'24.6%','backgroundColor':'#272e3e','display':'inline-block','marginRight':'10px'},
                                 children=[html.P(style={'textAlign':'center','fontSize':'3rem',
                                                         'fontWeight':'bold','color':'#a6d96a','padding':'1rem'},
                                                  children="Recovered Cases"),
                                           html.H3(style={'textAlign':'center','fontSize':'6rem',
                                                          'fontWeight':'bold','color':'#a6d96a'},
                                                   children='{}'.format(recoveredCases))]),
                             html.Div(style={'width':'24.6%','backgroundColor':'#272e3e','display':'inline-block','marginTop':'10px'},
                                  children=[html.P(style={'textAlign':'center','fontSize':'3rem',
                                                          'fontWeight':'bold','color':'#fdae61','padding':'1rem'},
                                                  children="Death Cases"),
                                            html.H3(style={'textAlign':'center','fontSize':'6rem',
                                                           'fontWeight':'bold','color':'#fdae61'},
                                                children='{}'.format(deathsCases))]),
                             html.Div(style={'width':'100%','marginTop':'10px'},
                                 children=[dcc.Graph(figure=fig2)]),
                             html.Div(style={'width':'100%','display':'inline-block','marginTop':'10px','marginBottom':'1.5%'},
                                  children=[dcc.Graph(figure=fig)])])
                 ]),
])


In [ ]:
if __name__ == '__main__':
    app.run_server(port=8882)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8882/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Jan/2020 18:00:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Jan/2020 18:00:11] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [31/Jan/2020 18:00:11] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [31/Jan/2020 18:00:11] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.1.2&m=1576595738 HTTP/1.1" 200 -
127.0.0.1 - - [31/Jan/2020 18:00:11] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.1&m=1576596177 HTTP/1.1" 200 -
127.0.0.1 - - [31/Jan/2020 18:00:11] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.3.1&m=1576595950 HTTP/1.1" 200 -
127.0.0.1 - - [31/Jan/2020 18:00:11] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.min.js?v=0.8.2&m=1580284437 HTTP/1.1